<a href="https://colab.research.google.com/github/sysadminamit/OnlineProductReviewNLP/blob/master/KerasEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving ElectronicsProductReviewClean.csv to ElectronicsProductReviewClean.csv


In [28]:
!ls

ElectronicsProductReviewClean.csv  sample_data


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import keras
from sklearn.utils import resample

In [0]:
df = pd.read_csv('ElectronicsProductReviewClean.csv')

In [31]:
data = df.sample(n = 5000) 
data = data[['Helpfulfactor','review']]
data.head()

,Helpfulfactor,review
66371,1.0,whats know seagate great brand product lot fas...
90467,1.0,camera fit perfectly add additional battery me...
31003,1.0,order bridge base product manual linksys web s...
51289,1.0,buy product motherinlaw would opportunity play...
18299,1.0,work great trooper put lot plug unplugs coil


In [0]:
data = data.dropna()

In [33]:
data['Helpfulfactor'].value_counts()

1.0    3500
0.0    1500
Name: Helpfulfactor, dtype: int64

In [34]:
from string import digits, punctuation
remove_set = digits + punctuation
with open("ElectronicsProductReviewClean.csv") as fin:
    words = {word.lower().strip(remove_set) for line in fin
         for word in line.rsplit(",",1)[0].split()}
    
print(len(words))

176069


In [35]:
data_majority = data[data.Helpfulfactor == 1 ]
data_minority = data[data.Helpfulfactor == 0 ]
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=3500,    # to match majority class
                                 random_state=123) # reproducible results
data = pd.concat([data_majority, data_minority_upsampled])
data['Helpfulfactor'].value_counts()

0.0    3500
1.0    3500
Name: Helpfulfactor, dtype: int64

In [0]:
max_fatures = 176069
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['review'].values)
X = tokenizer.texts_to_sequences(data['review'].values)
X = pad_sequences(X)

In [37]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
#model.add(BatchNormalization())
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.2, kernel_initializer=keras.initializers.he_uniform(seed=None)))
model.add(Dense(10,activation='relu', kernel_initializer=keras.initializers.he_uniform(seed=None)))
#model.add(Dropout(0.2))
model.add(Dense(2,activation='sigmoid', kernel_initializer=keras.initializers.he_uniform(seed=None)))
#model.add(BatchNormalization())
model.compile(loss = 'binary_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 783, 128)          22536832  
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1970      
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 22        
Total params: 22,793,624
Trainable params: 22,793,624
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
Y = pd.get_dummies(data['Helpfulfactor']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5600, 783) (5600, 2)
(1400, 783) (1400, 2)


In [42]:
batch_size = 256
model.fit(X_train, Y_train, epochs = 4, batch_size=batch_size, verbose = 1)

Epoch 1/4
5600/5600 [==============================] - 257s 46ms/step - loss: 0.0313 - acc: 0.9896
Epoch 2/4
5600/5600 [==============================] - 256s 46ms/step - loss: 0.0191 - acc: 0.9942
Epoch 3/4
5600/5600 [==============================] - 258s 46ms/step - loss: 0.0187 - acc: 0.9946
Epoch 4/4
5600/5600 [==============================] - 259s 46ms/step - loss: 0.0228 - acc: 0.9905


In [43]:
validation_size = 1000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.00
acc: 0.77


In [44]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 62.79527559055118 %
neg_acc 89.02439024390245 %


In [46]:
rvw = ['mount need strong sturdy fold almost flat enable television sit square flat wall desire extends rotates angle turn perfect fit long match correct visa pattern television']
#vectorizing the tweet by the pre-fitted tokenizer instance
rvw = tokenizer.texts_to_sequences(rvw)
#padding the tweet to have exactly the same shape as `embedding_2` input
rvw = pad_sequences(rvw, maxlen=783, dtype='int32', value=0)
print(rvw)
sentiment = model.predict(rvw,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("Not Helpful")
elif (np.argmax(sentiment) == 1):
    print("Hellpful")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [0]:
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'OnlineProductReviewHelpfulfactor.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

print('Saved trained model at %s ' % model_path)

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'OnlineProductReviewHelpfulfactor.h5',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/content/saved_models/OnlineProductReviewHelpfulfactor.h5', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))